In [6]:
import sys
sys.executable

'/n/groups/datta/tim_sainburg/conda_envs/openmmlab/bin/python3'

In [1]:
# module load cuda/11.7

In [2]:
# export TENSORRT_DIR=$/n/groups/datta/tim_sainburg/tensorrt/TensorRT-8.2.3.0
# export LD_LIBRARY_PATH=${TENSORRT_DIR}/lib:$LD_LIBRARY_PATH

In [3]:
import mmengine

In [5]:
from mmdeploy_runtime import Detector, PoseDetector
from mmdeploy_runtime import PoseTracker # not used here, but this runs both simultaneously
from mmpipeline.mmpose.skeletons.sainburg25pt import dataset_info 

ModuleNotFoundError: No module named 'mmdeploy_runtime'

In [5]:
#import tensorrt
#import pycuda

In [6]:
# create object detector
detector = Detector(
    #model_path="/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/tensortt/rtmdetection_fp32_Tesla_V100S-PCIE-32GB-0__24-08-07-15-53-50_038653", device_name="cuda"
    model_path="/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt-2/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB/", device_name="cuda"
)

[2024-08-07 16:30:22.560] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt-2/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB/"
[2024-08-07 16:30:23.099] [mmdeploy] [error] [trt_net.cpp:28] TRTNet: 1: [stdArchiveReader.cpp::StdArchiveReader::30] Error Code 1: Serialization (Serialization assertion magicTagRead == magicTag failed.Magic tag does not match)
[2024-08-07 16:30:23.099] [mmdeploy] [error] [trt_net.cpp:28] TRTNet: 4: [runtime.cpp::deserializeCudaEngine::50] Error Code 4: Internal Error (Engine deserialization failed.)
[2024-08-07 16:30:23.099] [mmdeploy] [error] [trt_net.cpp:75] failed to deserialize TRT CUDA engine
[2024-08-07 16:30:23.102] [mmdeploy] [error] [net_module.cpp:54] Failed to create Net backend: tensorrt, config: {
  "context": {
    "device": "<any>",
    "model": "<any>",
    "stream": "<any>"
  },
  "input": [
    "prep_output"
  ],
  "inpu

In [9]:
import numpy as np

In [ ]:
#detector(np.zeros((100,100,3)))

##### Duration: ???

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


In [3]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [4]:
import requests
import pandas as pd
from io import BytesIO

In [5]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [6]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,calibration_board_shape,calibration_square_size,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,NaN,NaN,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150


### Run 2D prediction

In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [9]:
output_directory_2D = output_directory/ "2D_predictions" 

In [10]:
output_directory_2D.mkdir(parents=True, exist_ok=True)

In [11]:
recording_row.video_location_on_o2

'/n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/'

In [12]:
video_directory = Path(recording_row.video_location_on_o2) / recording_row.video_recording_id

In [13]:
pose_checkpoint = "/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth"
pose_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py'
det_checkpoint = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth'
det_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py'

In [14]:
tensorrt_path = "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt"
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'

In [15]:
inferencer = Inferencer2D(
    video_directory,
    output_directory = output_directory_2D / recording_row.video_recording_id,
    pose_estimator_config = pose_config,
    pose_estimator_checkpoint = pose_checkpoint,
    detector_config = det_config,
    detector_checkpoint = det_checkpoint,
    tensorrt_rtmdetection_model_name = rtmdetection_model_name,
    tensorrt_rtmpose_model_name = rtmpose_model_name,
    tensorrt_model_directory = tensorrt_path,
    use_tensorrt = True,
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using CUDA device: Tesla_V100S-PCIE-32GB
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using tensorrt, loading models
loading libmmdeploy_trt_net.so ...
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Init completed
failed to load library libmmdeploy_trt_net.so
loading libmmdeploy_ort_net.so ...


### Load pose models

In [21]:
from motpy import Detection, MultiObjectTracker
from mmdet.apis import init_detector, inference_detector
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.apis import inference_topdown, init_model as init_pose_estimator

In [22]:
pose_estimator = init_pose_estimator(
    inferencer.pose_estimator_config,
    inferencer.pose_estimator_checkpoint,
    device="cuda",
    cfg_options=dict(model=dict(test_cfg=dict(output_heatmaps=False))),
)

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth


In [23]:
detector = init_detector(
    inferencer.detector_config, inferencer.detector_checkpoint, device="cuda"
)

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth


### Load tensortt models

In [ ]:
tensortt_detector = Detector(model_path='/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB', device_name="cuda")

In [21]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB

deploy.json  end2end.engine  output_pytorch.jpg   pipeline.json
detail.json  end2end.onnx    output_tensorrt.jpg


In [20]:
inferencer.tensorrt_detection_model_path.as_posix()

'/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB'

In [18]:
tensortt_detector = inferencer.Detector(
    model_path=inferencer.tensorrt_detection_model_path.as_posix(), device_name="cuda"
)

[2024-08-07 15:42:26.954] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/Tesla_V100S-PCIE-32GB"
[2024-08-07 15:42:26.955] [mmdeploy] [error] [net_module.cpp:47] Net backend not found: tensorrt, available backends: [("onnxruntime", 0)]
[2024-08-07 15:42:26.955] [mmdeploy] [error] [task.cpp:99] error parsing config: {
  "context": {
    "device": "<any>",
    "model": "<any>",
    "stream": "<any>"
  },
  "input": [
    "prep_output"
  ],
  "input_map": {
    "img": "input"
  },
  "is_batched": false,
  "module": "Net",
  "name": "rtmdet",
  "output": [
    "infer_output"
  ],
  "output_map": {},
  "type": "Task"
}


In [19]:
tensortt_pose_estimator = inferencer.PoseDetector(
    model_path=inferencer.tensorrt_pose_estimator_path.as_posix(), device_name="cuda"
)

[2024-08-07 15:42:27.358] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/Tesla_V100S-PCIE-32GB"
[2024-08-07 15:42:27.359] [mmdeploy] [error] [net_module.cpp:47] Net backend not found: tensorrt, available backends: [("onnxruntime", 0)]
[2024-08-07 15:42:27.359] [mmdeploy] [error] [task.cpp:99] error parsing config: {
  "context": {
    "device": "<any>",
    "model": "<any>",
    "stream": "<any>"
  },
  "input": [
    "prep_output"
  ],
  "input_map": {
    "img": "input"
  },
  "is_batched": true,
  "module": "Net",
  "name": "topdownposeestimator",
  "output": [
    "infer_output"
  ],
  "output_map": {},
  "type": "Task"
}


### Test run

In [28]:
inferencer.video_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846')

In [34]:
inferencer.all_videos = list(inferencer.video_directory.glob("*.mp4"))

In [36]:
video_path = inferencer.all_videos[0]

In [57]:
import cv2
import numpy as np
from tqdm.auto import tqdm

In [50]:
detection_interval=5
n_animals=1
total_frames=None
use_motpy=True
n_motpy_tracks=3
use_tensorrt=False
n_keypoints = 25

In [51]:
video = cv2.VideoCapture(video_path.as_posix())

In [52]:
total_frames = 90000

In [53]:
if total_frames is None:
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
if total_frames < 0:
    logger.info(f"Manually counting frames for {video_path}")
    # count frames manually
    total_frames = 0
    with tqdm(desc="counting frames") as pbar:
        while True:
            ret, frame = video.read()
            if not ret:
                break
            total_frames += 1
            pbar.update(1)
    logger.info(f"Total frames: {total_frames}")
    # release and reallocate video
    video.release()
    video = cv2.VideoCapture(video_path.as_posix())


In [54]:
# allocate arrays
detection_coords = np.zeros((total_frames, n_animals, 4))
detection_conf = np.zeros((total_frames, n_animals, 1))
keypoint_coords = np.zeros((total_frames, n_animals, n_keypoints, 2))
keypoint_conf = np.zeros((total_frames, n_animals, n_keypoints))

In [55]:
tracker = MultiObjectTracker(dt=0.1)
animal_ids = {str(i): i for i in range(n_animals)}
bboxes_motpy = np.zeros((n_animals, 4))
confs_motpy = np.zeros((n_animals, 1))
detection_changes = np.zeros((total_frames, n_animals, 1))
missing_detections = np.zeros((total_frames, n_animals, 1))

In [58]:
n_frames = 0
for frame_id in tqdm(range(total_frames), leave=False):
    success, frame = video.read()
    if not success:
        break
    break

  0%|          | 0/90000 [00:00<?, ?it/s]

In [59]:
result = inference_detector(detector, frame)

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201336/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [60]:
result

<DetDataSample(

    META INFORMATION
    ori_shape: (1200, 1220)
    img_id: 0
    scale_factor: (0.5245901639344263, 0.525)
    img_shape: (640, 640)
    img_path: None
    batch_input_shape: (640, 640)
    pad_shape: (640, 640)

    DATA FIELDS
    ignored_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            bboxes: tensor([], device='cuda:0', size=(0, 4))
            labels: tensor([], device='cuda:0', dtype=torch.int64)
        ) at 0x7ff141d0bc10>
    pred_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            scores: tensor([0.9092, 0.0210, 0.0201, 0.0194, 0.0178, 0.0126, 0.0122, 0.0112, 0.0105,
                        0.0103, 0.0100, 0.0100, 0.0095, 0.0094, 0.0093, 0.0090, 0.0090, 0.0084,
                        0.0083, 0.0083, 0.0082, 0.0081, 0.0080, 0.0080, 0.0078, 0.0077, 0.0076,
                        0.0074, 0.0072, 0.0072, 0.0071, 0.0070, 0.0070, 0.0070, 0.0069, 

In [61]:
bboxes = result.pred_instances["bboxes"][:n_motpy_tracks].cpu().numpy()
conf = result.pred_instances["scores"][:n_motpy_tracks].cpu().numpy()

In [62]:
bboxes

array([[ 846.08374,  398.01083, 1081.8966 ,  667.3192 ],
       [ 784.1225 ,  493.80035, 1100.8865 ,  859.4259 ],
       [ 716.9501 ,  381.5718 , 1052.2632 ,  671.5124 ]], dtype=float32)

In [63]:
conf

array([0.90918607, 0.02103002, 0.02012494], dtype=float32)

In [ ]:
test = tensortt_detector(frame)

In [1]:
test

NameError: name 'test' is not defined

### Test run

In [13]:
inferencer.run()

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth
Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth


  0%|          | 0/12 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Manually counting frames for /n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846.BackBottom.0.mp4
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Total frames: 90000


  0%|          | 0/90000 [00:00<?, ?it/s]

07/31 17:08:44 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".


...

  0%|          | 0/9 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Manually counting frames for /n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846.FrontRight.0.mp4
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Total frames: 90000


  0%|          | 0/90000 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Manually counting frames for /n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846.FrontRight.90000.mp4
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Total frames: 9


  0%|          | 0/9 [00:00<?, ?it/s]